In [92]:
import newspaper
from newspaper import Article

from newspaper3k_fixes import get_authors, get_keywords

import math

In [14]:
# A variety of articles with similar/different viewpoints and topics
urls = [
    # (CNN) Breaking news report on the Supreme Court's DACA decision
    'https://www.cnn.com/2020/06/18/politics/daca-immigration-supreme-court/index.html',

    # (Fox News) Report on Biden's statement on the decision, which he strongly celebrated
    'https://www.foxnews.com/politics/biden-calls-supreme-court-daca-ruling-a-victory-vows-to-make-program-permanent',

    # (Fox News) Fox News columnist calling the decision a 'travesty'
    'https://www.foxnews.com/media/guy-benson-supreme-court-daca-decision-travesty',

    # (The Heritage Foundation) From a conservative foundation, arguing that DACA is unconstitutional
    'https://www.heritage.org/courts/commentary/its-time-end-daca-its-unconstitutional-unless-approved-congress',

    # (MSNBC) Report on Trump's reaction, described as 'lashing out' after it doesn't 'go his way'
    'https://www.nbcnews.com/politics/donald-trump/trump-lashes-out-supreme-court-after-daca-ruling-doesn-t-n1231438',
]

articles = list(map(Article, urls))
for article in articles:
    Article.download(article)
    article.parse()
    
print("Titles:\n")
for i, article in enumerate(articles):
    print(f"{i} : {article.title}")

Titles:

0 : Supreme Court blocks Trump from ending DACA
1 : Biden calls Supreme Court DACA ruling a ‘victory,’ vows to make program ‘permanent’
2 : Guy Benson rips Supreme Court DACA decision as a 'travesty that could set a bad precedent'
3 : It’s Time to End DACA – It’s Unconstitutional Unless Approved by Congress
4 : Trump lashes out at Supreme Court after DACA ruling doesn't go his way


In [17]:
keywords = {}
for article in articles:
    keywords[article] = get_keywords(article)

In [26]:
# Split title on keywords

for article in articles:
    split_title = []

    acc = []
    for word in article.title.lower().split(" "):
        if word in keywords[article]['text']:
            if acc != []:
                split_title.append(" ".join(acc))
            split_title.append(word)
            acc = []
        else:
            acc.append(word)
            
    if(acc != []):
        split_title.append(" ".join(acc))

    print(split_title)

['supreme', 'court', 'blocks', 'trump', 'from ending', 'daca']
['biden calls', 'supreme', 'court', 'daca', 'ruling', 'a ‘victory,’ vows to make', 'program', '‘permanent’']
['guy', 'benson', 'rips supreme court', 'daca', 'decision', "as a 'travesty that could set a bad precedent'"]
['it’s time to end', 'daca', '– it’s unconstitutional unless approved by', 'congress']
['trump', 'lashes out at', 'supreme', 'court', 'after', 'daca', 'ruling', "doesn't go his way"]


In [113]:
# Finding proper nouns
article = articles[1]
title_words = article.title.split(" ")
print(f"words in the title: {title_words}\n")

# Just looking for anything that starts with a capital letter will 
# also catch the first word in a headline/sentence.
uppers = list(filter(lambda w: w[0].isupper(), title_words))
print(f"start w/ uppercase in title: {uppers}\n")

# So maybe look in the body and see if the first word in the headline
# is usually capitalized or not. Exclude words at the start of sentences.
body_words = {}
for sentence in article.text.split('.'):
    sentence = sentence.strip('\n').replace(',','').split(' ')
    
    if '' in sentence:
        sentence.remove('')
        
    for word in sentence[1:]:
        if word not in body_words:
            body_words[word] = {'total': 0, 'starts_with_cap': 0}
        body_words[word]['total'] += 1 
        if word[0].isupper():
            body_words[word]['starts_with_cap'] += 1
        
propers = []
for word in body_words:
    if body_words[word]['starts_with_cap'] > math.ceil(body_words[word]['total'] / 2):
        propers.append(word)
        
print(f"usually start w/ uppercase in body: {propers}\n")

# Combining multiple proper nouns in a row
# There are cases where this totally fails
split_title = []
combo = []
for i, word in enumerate(title_words):
    if word in propers:
        combo.append(word)
    elif word not in propers or i == len(title_words):
        if(combo != []):
            split_title.append(" ".join(combo))
            combo = []
        split_title.append(word)

print(f"combining propers: {split_title}")

words in the title: ['Biden', 'calls', 'Supreme', 'Court', 'DACA', 'ruling', 'a', '‘victory,’', 'vows', 'to', 'make', 'program', '‘permanent’']

start w/ uppercase in title: ['Biden', 'Supreme', 'Court', 'DACA']

usually start w/ uppercase in body: ['President', 'Biden', 'Supreme', 'Court', 'Trump', 'DACA', 'COURT', 'I']

combining propers: ['Biden', 'calls', 'Supreme Court DACA', 'ruling', 'a', '‘victory,’', 'vows', 'to', 'make', 'program', '‘permanent’']


In [68]:
# The Natural Language Toolkit (NLK)
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/blake/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [42]:
words = ['amazing', 'interesting', 'great', 'blocks']

for w in words:
    tmp = wn.synsets(w)[0].pos()
    print(w + ":" + tmp)

amazing:v
interesting:v
great:n
blocks:n


['', 'hello', 'from', 'sdfsd', '']
